## 전처리: 자연어의 노이즈 제거

### 1) 문장부호
- Hi, 가 Hi 와 , 의 결합인 것을 알지만 컴퓨터는 명시해 주지 않는다면 알파벳에 , 가 포함되어 있다고 생각할 수 있음
- 문장부호를 단어와 분리하면 해결이 되는 상황이기 때문에 문장부호 양쪽에 공백을 추가하는 방법

In [1]:
def pad_punctuation(sentence, punc):
    for p in punc:
        sentence = sentence.replace(p, " " + p + " ")
    return sentence

sentence = "Hi, my name is john."

print(pad_punctuation(sentence, [".", "?", "!", ","]))

Hi ,  my name is john . 


### 2) 대소문자
- First와 first 는 같은 의미를 갖고 있음에도 컴퓨터는 f와 F를 다르다고 구분지을 수 있음
- 모든 단어를 소문자로 바꾸는 방법

In [2]:
sentence = "First, open the first chapter."

print(sentence.lower())

first, open the first chapter.


In [3]:
sentence = "First, open the first chapter."

print(sentence.upper())

FIRST, OPEN THE FIRST CHAPTER.


### 3) 특수문자
- re 패키지(정규표현식 사용을 도와주는 패키지)를 이용하여 처리
- ten-year-old와 seven-year-old와 같은 표현들 사이에 있는 '-'를 처리

In [4]:
import re

sentence = "He is a ten-year-old boy."
sentence = re.sub("([^a-zA-Z.,?!])", " ", sentence)

print(sentence)

He is a ten year old boy.


In [5]:
# From The Project Gutenberg
# (https://www.gutenberg.org/files/2397/2397-h/2397-h.htm)

corpus = \
"""
In the days that followed I learned to spell in this uncomprehending way a great many words, among them pin, hat, cup and a few verbs like sit, stand and walk. 
But my teacher had been with me several weeks before I understood that everything has a name.
One day, we walked down the path to the well-house, attracted by the fragrance of the honeysuckle with which it was covered. 
Some one was drawing water and my teacher placed my hand under the spout. 
As the cool stream gushed over one hand she spelled into the other the word water, first slowly, then rapidly. 
I stood still, my whole attention fixed upon the motions of her fingers. 
Suddenly I felt a misty consciousness as of something forgotten—a thrill of returning thought; and somehow the mystery of language was revealed to me. 
I knew then that "w-a-t-e-r" meant the wonderful cool something that was flowing over my hand. 
That living word awakened my soul, gave it light, hope, joy, set it free! 
There were barriers still, it is true, but barriers that could in time be swept away.
""" 

def cleaning_text(text, punc, regex):
    # 노이즈 유형 (1) 문장부호 공백추가
    for p in punc:
        text = text.replace(p, " " + p + " ")

    # 노이즈 유형 (2), (3) 소문자화 및 특수문자 제거
    text = re.sub(regex, " ", text).lower()

    return text

print(cleaning_text(corpus, [".", ",", "!", "?"], "([^a-zA-Z0-9.,?!\n])"))


in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it

## 분산표현(distributed representation) 
- 희소 표현과는 반대되는 의미를 가짐
    - 희소 표현은 단어를 고정된 크기의 벡터로 표현하지 않고, 이진화 또는 빈도수의 방식으로 표현하는 방식
    - 단어의 존재 유무를 나타내며, 벡터 공간 상의 거리 측정, 단어 간의 의미 관계 파악이 어려움
- 분산 표현은 하나의 단어를 여러 차원의 값으로 나타낸 것
- 단어 간의 거리 측정을 통해 단어 간의 의미의 관련성을 파악할 수 있음
- Embedding 레이어를 사용해서 각 단어가 몇 차원의 속성을 가질지 정의하는 방식으로 분산 표현을 구현

In [6]:
# 만약 100개의 단어를 256차원의 속성으로 표현하고 싶다면
import tensorflow as tf

embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=256)

### 코사인 유사도(Cosine Similarity)
- 워드 벡터끼리는 단어들 간의 의미적 유사도를 계산
- 희소 표현을 보이기 위해 코사인 유사도를 사용해 봄

In [7]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

word_1 = np.array([-1.0, 0.0, 0.0, 0.0])
word_2 = np.array([0.0, 0.0, 1.0, 0.5])

print('word_1과 word_2의 유사도 :',cos_sim(word_1, word_2))

word_1과 word_2의 유사도 : 0.0


## 토큰화
- 토큰(Token) : 문장을 어떤 기준으로 쪼개었을 때, 쪼개진 각 단어들을 의미
- 쪼개진 기준이 토근화(Tokenization)기법에 의해 정해짐

### 1) 공백 기반 토큰화
- 말그대로 공백을 기반으로 토큰화를 진행

In [8]:
corpus = \
"""
in the days that followed i learned to spell in this uncomprehending way a great many words ,  among them pin ,  hat ,  cup and a few verbs like sit ,  stand and walk .  
but my teacher had been with me several weeks before i understood that everything has a name . 
one day ,  we walked down the path to the well house ,  attracted by the fragrance of the honeysuckle with which it was covered .  
some one was drawing water and my teacher placed my hand under the spout .  
as the cool stream gushed over one hand she spelled into the other the word water ,  first slowly ,  then rapidly .  
i stood still ,  my whole attention fixed upon the motions of her fingers .  
suddenly i felt a misty consciousness as of something forgotten a thrill of returning thought  and somehow the mystery of language was revealed to me .  
i knew then that  w a t e r  meant the wonderful cool something that was flowing over my hand .  
that living word awakened my soul ,  gave it light ,  hope ,  joy ,  set it free !  
there were barriers still ,  it is true ,  but barriers that could in time be swept away . 
"""

tokens = corpus.split()

print("문장이 포함하는 Tokens:", tokens)

문장이 포함하는 Tokens: ['in', 'the', 'days', 'that', 'followed', 'i', 'learned', 'to', 'spell', 'in', 'this', 'uncomprehending', 'way', 'a', 'great', 'many', 'words', ',', 'among', 'them', 'pin', ',', 'hat', ',', 'cup', 'and', 'a', 'few', 'verbs', 'like', 'sit', ',', 'stand', 'and', 'walk', '.', 'but', 'my', 'teacher', 'had', 'been', 'with', 'me', 'several', 'weeks', 'before', 'i', 'understood', 'that', 'everything', 'has', 'a', 'name', '.', 'one', 'day', ',', 'we', 'walked', 'down', 'the', 'path', 'to', 'the', 'well', 'house', ',', 'attracted', 'by', 'the', 'fragrance', 'of', 'the', 'honeysuckle', 'with', 'which', 'it', 'was', 'covered', '.', 'some', 'one', 'was', 'drawing', 'water', 'and', 'my', 'teacher', 'placed', 'my', 'hand', 'under', 'the', 'spout', '.', 'as', 'the', 'cool', 'stream', 'gushed', 'over', 'one', 'hand', 'she', 'spelled', 'into', 'the', 'other', 'the', 'word', 'water', ',', 'first', 'slowly', ',', 'then', 'rapidly', '.', 'i', 'stood', 'still', ',', 'my', 'whole', 'attenti

### 2) 형태소 기반 토큰화
- 한국어 문장은 공백 기준으로 토큰화를 했다간 엉망진창의 단어들이 등장하는 것을 확인 할 수 있음
- 이를 해결 하기 위해 형태소 기반 토큰화를 진행함
- 한국어 형태소 분석기는 "KoNLPY"가 있음
    - https://konlpy-ko.readthedocs.io/ko/v0.4.3/  
    - 내부적으로 5가지의 형태소 분석 Class를 포함하고 있음
-  한국어 형태소 분석기를 사용하는 비교 실험 진행
    - 속도 측면에서 뛰어난 분석기 : mecab
    - 띄어쓰기나 오탈자에 강건한 분석기 : KOMORAN

In [9]:
from konlpy.tag import Hannanum,Kkma,Komoran,Mecab,Okt

**설치 방법**  
$ pip install konlpy  

$ cd ~/aiffel/text_preprocess  

$ git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git  

$ cd Mecab-ko-for-Google-Colab  

$ bash install_mecab-ko_on_colab190912.sh  

**만약 JVMNotFoundException 오류가 발생한다면**  

$ sudo apt update  

$ sudo apt install default-jre

In [ ]:
tokenizer_list = [Hannanum(),Kkma(),Komoran(),Mecab(),Okt()]

kor_text = '코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤 전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.'

for tokenizer in tokenizer_list:
    print('[{}] \n{}'.format(tokenizer.__class__.__name__, tokenizer.pos(kor_text)))

[Hannanum] 
[('코로나바이러스', 'N'), ('는', 'J'), ('2019년', 'N'), ('12월', 'N'), ('중국', 'N'), ('우한', 'N'), ('에서', 'J'), ('처음', 'M'), ('발생', 'N'), ('하', 'X'), ('ㄴ', 'E'), ('뒤', 'N'), ('전', 'N'), ('세계', 'N'), ('로', 'J'), ('확산', 'N'), ('되', 'X'), ('ㄴ', 'E'), (',', 'S'), ('새롭', 'P'), ('은', 'E'), ('유형', 'N'), ('의', 'J'), ('호흡기', 'N'), ('감염', 'N'), ('질환', 'N'), ('이', 'J'), ('ㅂ니다', 'E'), ('.', 'S')]


### 3) 사전에 없는 단어의 문제
- 공백 기반이나 형태소 기반의 토큰화 기법들은 의미를 가지는 단위로 토큰을 생성함
- 이 기법들의 경우 모든 단어를 처리할 수 없기 때문에 자주 등장한 상위 N개의 단어만 사용하고 나머지는 \<unk>같은 특수 토큰으로 치환함  

코로나바이러스는 2019년 12월 중국 우한에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.

→ \<unk>는 2019년 12월 중국 \<unk>에서 처음 발생한 뒤
전 세계로 확산된, 새로운 유형의 호흡기 감염 질환입니다.  

- 위와 같은 현상을 OOV(out-of-vocabualry)문제라고 함
- OOV와 같이 새로 등장한 단어에 대해 약한 모습을 해결하고자 Worldpiece Model(WPM)이 등장함
- WPM은 pre+view와 pre+dict와 같이 pre(미리, 이전의)의미를 이용해서 한 단어를 여러 개의 subword의 집합으로 보는 방법을 말함

### Byte Pair Encoding(BPE)
- 데이터에서 가장 많이 등장하는 바이트 쌍(Byte Pair) 을 새로운 단어로 치환하여 압축하는 작업을 반복하는 방식으로 동작
-  모든 단어를 문자(바이트)들의 집합으로 취급하여 자주 등장하는 문자 쌍을 합치면, 접두어나 접미어의 의미를 캐치할 수 있고, 처음 등장하는 단어는 문자(알파벳)들의 조합으로 나타내어 OOV 문제를 해결

aaabdaaabac # 가장 많이 등장한 바이트 쌍 "aa"를 "Z"로 치환합니다.  
→   
ZabdZabac   # "aa" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.  
Z=aa        # 그다음 많이 등장한 바이트 쌍 "ab"를 "Y"로 치환합니다.  
→   
ZYdZYac     # "ab" 총 두 개가 치환되어 4바이트를 2바이트로 압축하였습니다.  
Z=aa        # 여기서 작업을 멈추어도 되지만, 치환된 바이트에 대해서도 진행한다면  
Y=ab        # 가장 많이 등장한 바이트 쌍 "ZY"를 "X"로 치환합니다.  
→   
XdXac  
Z=aa  
Y=ab  
X=ZY       # 압축이 완료되었습니다!  

In [ ]:
import re, collections

# 임의의 데이터에 포함된 단어들입니다.
# 우측의 정수는 임의의 데이터에 해당 단어가 포함된 빈도수입니다.
vocab = {
    'l o w '      : 5,
    'l o w e r '  : 2,
    'n e w e s t ': 6,
    'w i d e s t ': 3
}

num_merges = 5

def get_stats(vocab):
    """
    단어 사전을 불러와
    단어는 공백 단위로 쪼개어 문자 list를 만들고
    빈도수와 쌍을 이루게 합니다. (symbols)
    """
    pairs = collections.defaultdict(int)
    
    for word, freq in vocab.items():
        symbols = word.split()

        for i in range(len(symbols) - 1):             # 모든 symbols를 확인하여 
            pairs[symbols[i], symbols[i + 1]] += freq  # 문자 쌍의 빈도수를 저장합니다. 
        
    return pairs

def merge_vocab(pair, v_in):
    """
    문자 쌍(pair)과 단어 리스트(v_in)를 입력받아
    각각의 단어에서 등장하는 문자 쌍을 치환합니다.
    (하나의 글자처럼 취급)
    """
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
        
    return v_out, pair[0] + pair[1]

token_vocab = []

for i in range(num_merges):
    print(">> Step {0}".format(i + 1))
    
    pairs = get_stats(vocab)
    best = max(pairs, key=pairs.get)  # 가장 많은 빈도수를 가진 문자 쌍을 반환합니다.
    vocab, merge_tok = merge_vocab(best, vocab)
    print("다음 문자 쌍을 치환:", merge_tok)
    print("변환된 Vocab:\n", vocab, "\n")
    
    token_vocab.append(merge_tok)
    
print("Merged Vocab:", token_vocab)

### Wordpiece Model(WPM)
- 구글에서 BPE를 변형해 제안한 알고리즘
- BPE에 대해 두 가지 차별성을 가짐
    - 공백 복원을 위해 단어의 시작 부분에 언더바 _ 를 추가.
    - 빈도수 기반이 아닌 가능도(Likelihood)를 증가시키는 방향으로 문자 쌍을 합침. (더 '그럴듯한' 토큰을 만들어냄)
- 조사, 어미 등의 활용이 많고 복잡한 한국어 같은 모델의 토크나이저로 WPM이 좋은 대안이 될 수 있음
- WPM은 어떤 언어든 무관하게 적용 가능한 language-neutral하고 general한 기법
- 한국어 형태소 분석기처럼 한국어에만 적용 가능한 기법보다 훨씬 활용도가 큼
- 구글의 SentencePiece 라이브러리를 통해 사용 가능
- SentencePiece에는 전처리 과정도 포함되어 있어서, 데이터를 따로 정제할 필요가 없음
- https://github.com/google/sentencepiece
    

### soynlp
- '학습데이터를 이용하지 않으면서 데이터에 존재하는 단어를 찾거나, 문장을 단어열로 분해, 혹은 품사 판별을 할 수 있는 비지도학습 접근법을 지향합니다'

-  soynlp는 한국어 자연어 처리를 위한 라이브러리
- 토크나이저 외에도 단어 추출, 품사 판별, 전처리 기능도 제공
- 형태소 기반인 koNLPy의 단점을 해결하기 위해 사용
- 단어의 경계를 비지도학습을 통해 결정 > 미등록 단어도 토큰화 가능
- 통계 기반 토크나이저로 분류하기도 함

## 토큰에게 의미를 부여하기
- 각 토큰들이 랜덤하게 부여된 실수로 살아가지 않게, 그들끼리 유사도 연산을 할 수 있게 의미를 부여하는 알고리즘

### Word2Vec
- "단어를 벡터로 만든다"
- 단어 벡터 간의 유의미한 유사도를 반영하여 단어의 의미를 수치화 함
- 학습 방식에는 CBOW(Continuous Bag of Words)와 Skip-Gram 두 가지 방식을 이용
    - CBOW: 주변의 단어들을 입력으로 중간에 있는 단어들을 예측
    - Skip-Gram: 중간에 있는 단어들을 입력으로 주변 단어들을 예측
    - Skip-Gram이 실제 실험에서 다소 우세한 경향이 있음
- 자주 등장하지 않는 단어는 최악의 경우 단 한 번의 연산만을 거쳐 랜덤하게 초기화된 값과 크게 다르지 않은 상태로 알고리즘이 종료한다는 단점이 있음

### FastText
- Word2Vec의 단점을 해결하기 위해 등장
- FastText는 한 단어를 n-gram의 집합이라고 보고 단어를 쪼개어 각 n-gram에 할당된 Embedding의 평균값을 사용
- https://brunch.co.kr/@learning/7

### ELMo - the 1st Contextualized Word Embedding
- 동음이의어를 처리할 수 없다는 단점을 처리하기 위해 등장
- 그 단어가 사용된 주변 단어의 맥락을 함께 고려 (contextualized word embedding)
- https://brunch.co.kr/@learning/12
- ELMo라는 모델은 첫 번째 Contextualized Word Embedding 모델